In [1]:
%matplotlib inline

import datetime as dt

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sn

import nivapy3 as nivapy

plt.style.use("ggplot")

# Biolok data analysis

## 1. Figures for the 2018 report

With help from Liv Bente, I have created a new "dataset" named `BIOLOK rapp2018`, which is accessible under `Datasets` in RESA's `Projects` window. The steps to create this dataset are described in *update_db_2018_report.ipynb*.

To export the relevant data from RESA2, select the dataset and remember to check `Use only water samples from selected projects` before extracting the data.

Øyvind would like a set of plots similar to the ones on pages 29 - 38 [here](https://www.miljodirektoratet.no/globalassets/publikasjoner/m503/m503.pdf) (see e-mail received 17.08.2020 at 15:34.

For now, I have manually extracted the relevant data from RESA and saved it to 

    biolok\biolok_data_2020-08-21.xlsx

In [2]:
# Read data
df = pd.read_excel("../biolok_data_2020-08-21.xlsx", sheet_name="data")

# Replace neagtive LAL with 0
df["LAL_ug/l"] = df["LAL_ug/l"].clip(lower=0)

# Add Ca and Mg
df["ECa-Mg_uekv/l"] = df["EMg_uekv/l"] + df["ECa_uekv/l"]
del df["EMg_uekv/l"], df["ECa_uekv/l"]

# Get region and site number
df[["region", "site_no"]] = df["biolok_code"].str.split("-", expand=True)

df["year"] = df["date"].dt.year

df.head()

,station_id,station_code,biolok_code,station_name,date,depth1,depth2,ESO4_uekv/l,ANC_uekv/l,pH,LAL_ug/l,TOC_mg-C/l,ECa-Mg_uekv/l,region,site_no,year
0,14,432-1-26,I-3,Måsabutjørna,1995-09-20,0.0,0.0,38.68637,6.63334,5.68,0.0,1.6,26.03949,I,3,1995
1,14,432-1-26,I-3,Måsabutjørna,1996-10-10,0.0,0.0,36.31385,3.69217,5.73,0.0,1.9,24.88327,I,3,1996
2,14,432-1-26,I-3,Måsabutjørna,1997-10-26,0.0,0.0,36.02333,6.46292,5.70,9.0,1.5,27.71908,I,3,1997
3,14,432-1-26,I-3,Måsabutjørna,1998-07-06,0.0,0.0,30.06786,5.48708,5.78,2.0,1.7,21.91635,I,3,1998
4,14,432-1-26,I-3,Måsabutjørna,1998-09-15,0.0,0.0,32.14986,4.58344,5.93,2.0,1.7,22.23999,I,3,1998


In [3]:
# Annual means by station
agg = df.groupby(["station_name", "year", "region"]).mean().reset_index()
del agg["station_id"], agg["depth1"], agg["depth2"]

In [4]:
label_dict = {
    "ESO4_uekv/l": "Ikke-marin $SO_4$ (µekv/l)",
    "ECa-Mg_uekv/l": "Ikke-marin Ca+Mg (µekv/l)",
    "ANC_uekv/l": "ANC (µekv/l)",
    "pH": "pH",
    "LAL_ug/l": "Labilt Al (µg/l)",
    "TOC_mg-C/l": "TOC (mg C/l)",
}

# Pars to plot
pars = ["ESO4_uekv/l", "ECa-Mg_uekv/l", "ANC_uekv/l", "pH", "LAL_ug/l", "TOC_mg-C/l"]

# Loop over regions
for reg in agg["region"].unique():
    reg_df = agg.query("region == @reg")
    del reg_df["region"]

    # Setup plot
    fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(16, 9))
    axes = axes.flatten()

    # Loop over pars
    #    max_list = []
    for idx, par in enumerate(pars):
        par_df = reg_df[["station_name", "year", par]].copy()
        par_df.set_index(["station_name", "year"], inplace=True)
        par_df = par_df.unstack("station_name")
        par_df.columns = par_df.columns.get_level_values(1)

        if len(par_df.columns) > 1:
            par_df["Snitt"] = par_df.mean(axis=1, skipna=False)

        #        if par in ["ESO4_uekv/l", "ECa-Mg_uekv/l", "ANC_uekv/l"]:
        #            max_list.append(np.nanmax(par_df.values))

        # Plot each station
        for stn in par_df.columns:
            if stn == "Snitt":
                par_df[stn].dropna().plot(ax=axes[idx], style="-", lw=3, c="k")
            else:
                par_df[stn].dropna().plot(ax=axes[idx], style="--", lw=2)

        #        if par == 'pH':
        #            axes[idx].set_ylim((4, 7))
        axes[idx].set_xlim((1985, 2020))
        axes[idx].set_xlabel("")
        axes[idx].set_ylabel(label_dict[par])

    #    for ax in [0, 1]:
    #        axes[ax].set_ylim((0, max(max_list)))
    #    axes[2].set_ylim(ymax=max(max_list))

    axes[-1].legend(loc="upper center", bbox_to_anchor=(0.5, -0.1), ncol=3)
    plt.tight_layout()

    out_png = f"./region_plots/water_chem_ts_region_{reg.lower()}.png"
    plt.savefig(out_png, dpi=300)
    plt.close()

## 2. Regional Mann-Kendall

In [5]:
# Pars to plot
pars = ["ESO4_uekv/l", "ECa-Mg_uekv/l", "ANC_uekv/l", "pH", "LAL_ug/l", "TOC_mg-C/l"]

res_list = []

# Loop over regions
for reg in agg["region"].unique():
    reg_df = agg.query("region == @reg")

    # Loop over pars
    for par in pars:
        res_df = nivapy.stats.seasonal_regional_mk_sen(
            reg_df, time_col="year", value_col=par, block_col="station_name"
        ).T.loc['value']
        res_df['region'] = reg
        res_df['parameter'] = par
        res_list.append(res_df)

res_df = pd.concat(res_list, axis=1, sort=False).T.reset_index(drop=True)
res_df = res_df[['region', 'parameter', 's', 'z', 'var_s', 'p', 'sslp', 'trend']]
res_df.to_csv('regional_mk_trends.csv', index=False)

res_df

,region,parameter,s,z,var_s,p,sslp,trend
0,I,ESO4_uekv/l,-991,-11.6353,7239.67,0,-0.84127,decreasing
1,I,ECa-Mg_uekv/l,-60,-0.693366,7240.67,0.48808,-0.0442908,no trend
2,I,ANC_uekv/l,705,8.27282,7241.67,2.22045e-16,1.12356,increasing
3,I,pH,657,7.71338,7233,1.22125e-14,0.0152632,increasing
4,I,LAL_ug/l,-8,-0.0826413,7174.67,0.934137,0,no trend
5,I,TOC_mg-C/l,387,4.54579,7210.33,5.47289e-06,0.0266667,increasing
6,IV,ESO4_uekv/l,-1679,-14.1516,14059.7,0,-2.06651,decreasing
7,IV,ECa-Mg_uekv/l,-711,-5.98785,14059.7,2.12634e-09,-0.511123,decreasing
8,IV,ANC_uekv/l,1315,11.0817,14059.7,0,1.94012,increasing
9,IV,pH,1137,9.5867,14041.7,0,0.0217647,increasing
